In [1]:
import logging
import os
import psutil
from typing import List
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
class RAGPipeline: 
    
    def __init__(self):
        self.setup_logging()
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        self.llm = Ollama(model="deepseek-r1:8b")
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        
    def setup_logging(self):
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        
    def load_and_split_documents(self, file_path: str) -> List[Document]:
        self.log_memory_usage("Before loading documents")
        loader = TextLoader(file_path)
        data = loader.load()
        self.log_memory_usage("After loading documents")
        documents = self.text_splitter.split_documents(data)
        self.log_memory_usage("After splitting documents")
        return documents
        
    def create_vector_store(self, documents: List[Document]) -> FAISS:
        db = FAISS.from_documents(documents, self.embeddings)
        self.log_memory_usage("After creating vector store")
        return db
        
    def setup_retrieval_chain(self, db: FAISS):
        prompt = ChatPromptTemplate.from_template("""
        Answer the following question based on the context provided:
        <context>
        {context}
        </context>
        Question: {input}
        """)
        
        document_chain = create_stuff_documents_chain(llm=self.llm, prompt=prompt)
        retriever = db.as_retriever()
        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        return retrieval_chain
        
    def query_model(self, retrieval_chain, query: str):
        self.log_memory_usage("Before executing query")
        response = retrieval_chain.invoke({"input": query})
        self.log_memory_usage("After executing query")
        return response['answer']

    def log_memory_usage(self, stage: str):
        process = psutil.Process(os.getpid())
        memory_info = process.memory_info()
        logging.info(f"{stage}: Memory usage - {memory_info.rss / (1024 * 1024):.2f} MB")

In [ ]:
if __name__ == "__main__":
    pipeline = RAGPipeline()
    file_path = "Docmid_report.txt"
    documents = pipeline.load_and_split_documents(file_path)
    db = pipeline.create_vector_store(documents) 
    retrieval_chain = pipeline.setup_retrieval_chain(db)
    
    query = "What is the function of compute_metrics?"
    answer = pipeline.query_model(retrieval_chain, query)
    print(f"Answer: {answer}")

/home/durga/Hw4/hw4/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2025-04-24 21:52:47,940 - INFO - Use pytorch device_name: cuda
2025-04-24 21:52:47,940 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2025-04-24 21:52:48,463 - INFO - Before loading documents: Memory usage - 728.00 MB
2025-04-24 21:52:48,464 - INFO - After loading documents: Memory usage - 728.00 MB
2025-04-24 21:52:48,465 - INFO - After splitting documents: Memory usage - 728.00 MB
2025-04-24 21:52:48,613 - INFO - Loading faiss with AVX2 support.
2025-04-24 21:52:48,614 - INFO - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2025-04-24 21:52:48,614 - INFO - Loading faiss.
2025-04-24 21:52:48,625 - INFO